In [1]:
from Mancala import Mancala
from MancalaAI import MancalaAI, get_alpha_beta_minimax_move, get_basic_minimax_move
import numpy as np
import time

# Simulating 100,000 rounds of random vs. random:

In [2]:
# simulating 100000 rounds of random vs random
num_sims = 100000
rand_p1_wins = 0
rand_p2_wins = 0
num_ties = 0
moves_played_arr = []
moves_to_win_arr = [] # only for games where there was a definitive winner (ignoring ties)
for _ in range(num_sims):
    game = Mancala(pits_per_player=6, stones_per_pit=4)
    moves = 0
    while not game.winning_eval():
        rand_move = game.random_move_generator()
        if rand_move is None:
            break
        game.play(rand_move)
        moves += 1
        
    # update win counters once every time a game is completed
    rand_p1 = game.board[game.p1_mancala_index]
    rand_p2 = game.board[game.p2_mancala_index]
    if rand_p1 > rand_p2:
        rand_p1_wins += 1
        moves_to_win_arr.append(moves)
    elif rand_p1 < rand_p2:
        rand_p2_wins += 1
        moves_to_win_arr.append(moves)
    else:
        num_ties += 1
        
    moves_played_arr.append(moves)

print(f"Number of ties: {num_ties}")
print(f"Random Player 1 wins: {rand_p1_wins} times.")
print(f"Random Player 2 wins: {rand_p2_wins} times.")
# print(moves_played_arr)
print(f"Average moves played to win: {np.mean(moves_to_win_arr)}") # avg moves to win
print(f"Average moves played overall: {np.mean(moves_played_arr)}")
# assert(num_ties + rand_p1_wins + rand_p2_wins == num_sims)
print(f"Player 1 wins {rand_p1_wins / num_sims * 100} % of the time.")
print(f"Player 2 wins {rand_p2_wins / num_sims * 100} % of the time.")
print(f"The players tied {num_ties / num_sims * 100} % of the time.")

Number of ties: 6576
Random Player 1 wins: 47924 times.
Random Player 2 wins: 45500 times.
Average moves played to win: 44.38042687103956
Average moves played overall: 44.58248
Player 1 wins 47.924 % of the time.
Player 2 wins 45.5 % of the time.
The players tied 6.576 % of the time.


## Simulating 100 rounds of Basic Minimax AI Player (plie_lim=5) vs Random Player:

In [3]:
num_sims = 100
ai_wins = 0
random_wins = 0
ties = 0
moves_played_arr = []
moves_to_win_arr = []
time_arr = []
move_time_arr = []

for _ in range(num_sims):
    game = MancalaAI(pits_per_player=6, stones_per_pit=4)
    state = game.initial # get initial state
    moves = 0
    start_time = time.time()
    
    while not game.terminal_test(state):
        current_player = game.to_move(state)
        
        if current_player == 1: ## the AI player should be player 1
            move_start_time = time.time()
            move = get_basic_minimax_move(game, state, plie_lim=5)
            move_end_time = time.time()
            move_time_arr.append(move_end_time - move_start_time)
        else:
            move = state.board.random_move_generator()
            
        if move is None:
            break
    
        state = game.result(state, move)
        moves += 1
        
    end_time = time.time()
    time_arr.append(end_time - start_time)
    
    # check winner and update counters
    final_mancala = state.board # grab the final board state
    ai_score = final_mancala.board[final_mancala.p1_mancala_index]
    random_score = final_mancala.board[final_mancala.p2_mancala_index]
    
    if ai_score > random_score:
        ai_wins += 1
        moves_to_win_arr.append(moves)
    elif ai_score < random_score:
        random_wins += 1
        moves_to_win_arr.append(moves)
    else:
        ties += 1
        
    moves_played_arr.append(moves)
    
print(f"AI Player wins: {ai_wins} ({ai_wins / num_sims * 100:.1f}%)")
print(f"Random Player wins: {random_wins} ({random_wins / num_sims * 100:.1f}%)")
print(f"Ties: {ties} ({ties/num_sims * 100:.1f}%)")
print(f"Average moves played: {np.mean(moves_played_arr):.1f}")
print(f"Average moves played to win: {np.mean(moves_to_win_arr):.1f}")
print(f"Average time per move for the AI player with basic minimax 5-ply depth: {np.mean(move_time_arr):.3f} seconds")
print(f"Average time per game: {np.mean(time_arr):.3f} seconds")

AI Player wins: 95 (95.0%)
Random Player wins: 3 (3.0%)
Ties: 2 (2.0%)
Average moves played: 29.2
Average moves played to win: 29.3
Average time per move for the AI player with basic minimax 5-ply depth: 0.239 seconds
Average time per game: 3.557 seconds


# Simulating 100 rounds of Minimax with Alpha-Beta Pruning AI Player (plie_lim=5) vs Random Player:

In [4]:
num_sims = 100
ai_wins = 0
random_wins = 0
ties = 0
moves_played_arr = []
moves_to_win_arr = []
time_arr = []
move_time_arr = []

for _ in range(num_sims):
    game = MancalaAI(pits_per_player=6, stones_per_pit=4)
    state = game.initial # get initial state
    moves = 0
    start_time = time.time()
    
    while not game.terminal_test(state):
        current_player = game.to_move(state)
        
        if current_player == 1: ## the AI player should be player 1
            move_start_time = time.time()
            move = get_alpha_beta_minimax_move(game, state, plie_lim=5)
            move_end_time = time.time()
            move_time_arr.append(move_end_time - move_start_time)
        else:
            move = state.board.random_move_generator()
            
        if move is None:
            break
    
        state = game.result(state, move)
        moves += 1
        
    end_time = time.time()
    time_arr.append(end_time - start_time)
        
    # check winner and update counters
    final_mancala = state.board # grab the final board state
    ai_score = final_mancala.board[final_mancala.p1_mancala_index]
    random_score = final_mancala.board[final_mancala.p2_mancala_index]
    
    if ai_score > random_score:
        ai_wins += 1
        moves_to_win_arr.append(moves)
    elif ai_score < random_score:
        random_wins += 1
        moves_to_win_arr.append(moves)
    else:
        ties += 1
        
    moves_played_arr.append(moves)
    
print(f"AI Player wins: {ai_wins} ({ai_wins / num_sims * 100:.1f}%)")
print(f"Random Player wins: {random_wins} ({random_wins / num_sims * 100:.1f}%)")
print(f"Ties: {ties} ({ties/num_sims * 100:.1f}%)")
print(f"Average moves played: {np.mean(moves_played_arr):.1f}")
print(f"Average moves played to win: {np.mean(moves_to_win_arr):.1f}")
print(f"Average time per move for the AI player with alpha beta pruning and 5-ply depth: {np.mean(move_time_arr):.3f} seconds")
print(f"Average time per game: {np.mean(time_arr):.3f} seconds")

AI Player wins: 97 (97.0%)
Random Player wins: 2 (2.0%)
Ties: 1 (1.0%)
Average moves played: 28.8
Average moves played to win: 28.9
Average time per move for the AI player with 5-ply depth: 0.036 seconds
Average time per game: 0.529 seconds


## Simulating 100 rounds of Minimax with Alpha-Beta Pruning AI player (plie_lim=10) vs Random Player:

In [ ]:
num_sims = 100
ai_wins = 0
random_wins = 0
ties = 0
moves_played_arr = []
moves_to_win_arr = []
time_arr = []
move_time_arr = []

for _ in range(num_sims):
    game = MancalaAI(pits_per_player=6, stones_per_pit=4)
    state = game.initial # get initial state
    moves = 0
    start_time = time.time()
    
    while not game.terminal_test(state):
        current_player = game.to_move(state)
        
        if current_player == 1: ## the AI player should be player 1
            move_start_time = time.time()
            move = get_alpha_beta_minimax_move(game, state, plie_lim=10)
            move_end_time = time.time()
            move_time_arr.append(move_end_time - move_start_time)
        else:
            move = state.board.random_move_generator()
            
        if move is None:
            break
    
        state = game.result(state, move)
        moves += 1
        
    end_time = time.time()
    time_arr.append(end_time - start_time)
        
    # check winner and update counters
    final_mancala = state.board # grab the final board state
    ai_score = final_mancala.board[final_mancala.p1_mancala_index]
    random_score = final_mancala.board[final_mancala.p2_mancala_index]
    
    if ai_score > random_score:
        ai_wins += 1
        moves_to_win_arr.append(moves)
    elif ai_score < random_score:
        random_wins += 1
        moves_to_win_arr.append(moves)
    else:
        ties += 1
        
    moves_played_arr.append(moves)
    
print(f"AI Player wins: {ai_wins} ({ai_wins / num_sims * 100:.1f}%)")
print(f"Random Player wins: {random_wins} ({random_wins / num_sims * 100:.1f}%)")
print(f"Ties: {ties} ({ties/num_sims * 100:.1f}%)")
print(f"Average moves played: {np.mean(moves_played_arr):.1f}")
print(f"Average moves played to win: {np.mean(moves_to_win_arr):.1f}")
print(f"Average time per move for the AI player with alpha-beta pruning and 10-ply depth: {np.mean(move_time_arr):.3f} seconds")
print(f"Average time per game: {np.mean(time_arr):.3f} seconds")